In [1]:
import torch as tn
import torchtt as tntt
import TTCME
import matplotlib.pyplot as plt 
import scipy.integrate
import numpy as np
import datetime

tn.set_default_tensor_type(tn.DoubleTensor)

In [2]:
r1 = TTCME.ChemicalReaction(['mRNA','protein'],'mRNA->mRNA+protein', 0.015)
r2 = TTCME.ChemicalReaction(['mRNA','protein'],'mRNA->', 0.002)
r3 = TTCME.ChemicalReaction(['mRNA','protein'],'->mRNA', 0.1)
r4 = TTCME.ChemicalReaction(['mRNA','protein'],'protein->', 0.01)

rs = TTCME.ReactionSystem(['mRNA','protein'],[r1, r2, r3, r4])

N = [80,120] 

Att = rs.generatorTT(N)

p0 = TTCME.pdf.SingularPMF(N,[2,4])

Nt = 8
dT = 128

In [3]:
Asp = rs.generator_sparse(N)

res = scipy.integrate.solve_ivp(lambda t,y: Asp.dot(y), [0,dT*Nt],p0.dofs.numpy().flatten(),max_step=dT/10000)
Pt = res.y.reshape(N+[-1])
p_ref = Pt[:,:,-1]

In [4]:

print('Implicit Euler...')
err_implicit = []
refinements_implicit = [16,32,64,128,256,512]

for nt in refinements_implicit:
    integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=nt, epsilon = 1e-9, dt_max = 100, method='implicit-euler')

    p = p0.copy()
    for i in range(Nt):
        p.dofs = integrator.solve(p.dofs, dT, intervals = 1)
        
    err = np.max(np.abs(p.dofs.numpy()-p_ref)) / np.max(np.abs(p_ref))
    err_implicit.append(err)
    print('nt ',nt,' error inf ',err)

Implicit Euler...
nt  16  error inf  0.019627063123447894
nt  32  error inf  0.009900226272254842
nt  64  error inf  0.004970279641294143
nt  128  error inf  0.0024898341568327714
nt  256  error inf  0.001246546336222805
nt  512  error inf  0.0006237325361299042


In [5]:

print('Crank Nicolson...')
err_cn = []
refinements_cn = [16,32,64,128,256,512]

for nt in refinements_cn:
    integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=nt, epsilon = 1e-11, dt_max = 100, method='crank–nicolson')

    p = p0.copy()
    for i in range(Nt):
        p.dofs = integrator.solve(p.dofs, dT, intervals = 1)
        
    err = np.max(np.abs(p.dofs.numpy()-p_ref)) / np.max(np.abs(p_ref))
    err_cn.append(err)
    print('nt ',nt,' error inf ',err)

Crank Nicolson...
nt  16  error inf  0.00018992100475586157
nt  32  error inf  4.443183395997286e-05
nt  64  error inf  1.0756222728666026e-05
nt  128  error inf  2.646697657542131e-06
nt  256  error inf  6.564063402092687e-07
nt  512  error inf  1.6228106978344783e-07


In [6]:

print('Cheby...')
err_ch = []
refinements_ch = [2,6,8,10,12,14,16,18,20,22,24,28,32]

for nt in refinements_ch:
    integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=nt, epsilon = 1e-14, dt_max = 100, method='cheby')

    p = p0.copy()
    for i in range(Nt):
        p.dofs = integrator.solve(p.dofs, dT, intervals = 1)
        
    err = np.max(np.abs(p.dofs.numpy()-p_ref)) / np.max(np.abs(p_ref))
    err_ch.append(err)
    print('nt ',nt,' error inf ',err)

Cheby...
nt  2  error inf  0.006089530433511648
nt  6  error inf  7.520282605165915e-11


_LinAlgError: linalg.svd: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 7).

In [ ]:
print('Legendre...')
err_le = []
refinements_le = [2,6,8,10,12,14,16,18,20,22,24,28,32]

for nt in refinements_le:
    integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=nt, epsilon = 1e-14, dt_max = 100, method='legendre')

    p = p0.copy()
    for i in range(Nt):
        p.dofs = integrator.solve(p.dofs, dT, intervals = 1)
        
    err = np.max(np.abs(p.dofs.numpy()-p_ref)) / np.max(np.abs(p_ref))
    err_le.append(err)
    print('nt ',nt,' error inf ',err)

In [ ]:
plt.figure()
plt.loglog(refinements_implicit,err_implicit)
plt.loglog(refinements_cn[:-1],err_cn[:-1])
plt.loglog(refinements_ch[:],err_ch[:])
# plt.loglog(refinements_le[:],err_le[:])
plt.xlabel(r'$N_t$')
plt.ylabel(r'max relative error')
plt.grid()
plt.legend(['Implicit Euler','Crank-Nicolson','Chebyshev'])

In [ ]:
print('Epsilon of the solver...')
err_eps = []
refinements_epsilon = 10.0 ** (-np.arange(1,11))
for eps in refinements_epsilon:
    integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=nt, epsilon = eps, dt_max = 100, method='cheby')

    p = p0.copy()
    for i in range(Nt):
        p.dofs = integrator.solve(p.dofs, dT, intervals = 1)
        
    err = np.max(np.abs(p.dofs.numpy()-p_ref)) / np.max(np.abs(p_ref))
    err_eps.append(err)
    print('epsilon ',eps,' error inf ',err)

In [ ]:
print('Epsilon vs Nt ...')
refinements_epsilon_2 = 10.0 ** (-np.arange(1,13))
refinements_ch2 = [2,3,4,5,6,7,8]
err_eps_ch = []
for eps in refinements_epsilon_2:
    err_temp = []
    for nt in refinements_ch2:
        integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=nt, epsilon = eps, dt_max = 100, method='cheby')
        p = p0.copy()
        for i in range(Nt):
            p.dofs = integrator.solve(p.dofs, dT, intervals = 1)
        
        err = np.max(np.abs(p.dofs.numpy()-p_ref)) / np.max(np.abs(p_ref))
        err_temp.append(err)
        print('epsilon ',eps,' nt ',nt,' error inf ',err)
    err_eps_ch.append(err_temp)

In [ ]:
plt.figure()
plt.loglog(refinements_epsilon,err_eps)
plt.xlabel(r'$\epsilon$')
plt.ylabel(r'max relative error')
plt.grid()
# tikzplotlib.save('convergence_eps.tex')

plt.figure()
plt.loglog(dT/np.array(refinements_ch2),np.array(err_eps_ch).transpose())
plt.xlabel(r'$\Delta t$ [s]')
plt.ylabel(r'max relative error')
plt.legend([r'$\epsilon=$'+str(eps) for eps in refinements_epsilon_2])
plt.grid()
# tikzplotlib.save('convergence_eps_multiple.tex')

plt.figure()
plt.loglog(np.array(refinements_epsilon_2),np.array(err_eps_ch))
plt.xlabel(r'$\epsilon$')
plt.ylabel(r'max relative error')
plt.legend([r'$T=$'+str(tmp)+'' for tmp in np.array(refinements_ch2)])
plt.grid()
# tikzplotlib.save('convergence_Nt_multiple.tex')